In [1]:
import numpy as np
import matplotlib as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
from os import listdir
import os
import glob

import xml.etree.ElementTree as ET

# tmp = keras.utils.load_img(
#             'dataset/images/road0.png',
#             color_mode = 'grayscale',
#             target_size=(180, 180)
#         )

In [2]:
model_big = keras.models.load_model('model_huge.keras')
model_medium = keras.models.load_model('model_medium.keras')
model_small = keras.models.load_model('model_small.keras')

In [3]:
labels_path = 'dataset/annotations/'
img_path = 'dataset/images/'

label_files = sorted(listdir(labels_path))
img_files = sorted(listdir(img_path))

cathegories = ['trafficlight', 'stop', 'speedlimit', 'crosswalk']

assert len(label_files) == len(img_files) # sprawdzenie, czy ilość zdjęć pokrywa się z ilością labeli

for i in range(len(label_files)): # sprawdzenie, czy nazwy plików i zdjęć są w odpowiedniej kolejności
    assert(label_files[i][:-4] == img_files[i][:-4])

def get_class_vector(classes): 
    tmp = [0, 0, 0, 0]

    for i in range(4):
        if(cathegories[i] in classes):
            tmp[i] = 1
    
    return tmp


label_vectors = []

for i in range(len(label_files)):
    xml_data = open('dataset/annotations/' + label_files[i], 'r').read()
    root = ET.XML(xml_data)
    
    data = []
    
    for i in root.findall('object'):
        c = i.find('name').text
        data.append(c)
    label_vectors.append(get_class_vector(data))

img_vector = []

for i in range(len(img_files)):
    img_vector.append(np.array(
        keras.utils.load_img(
            img_path + img_files[i],
            color_mode = 'grayscale',
            target_size=(180, 180)
        )).astype("float32")
    )

In [12]:
in_values = np.array(img_vector[-200:])
targets = np.array(label_vectors[-200:])

# predict_big = model_big.predict(in_values)
# predict_medium = model_medium.predict(in_values)
# predict_small = model_small.predict(in_values)

model_small.evaluate(in_values, targets)
model_medium.evaluate(in_values, targets)
model_big.evaluate(in_values, targets)

# def evaluate(pred, targets):
#     return np.sum(np.abs(pred - targets)) / len(pred)

# print(f'error big model: {evaluate(predict_big, targets)}\nerror medium model: {evaluate(predict_medium, targets)}\nerror small model: {evaluate(predict_small, targets)}')

7/7 [==============================] - 5s 694ms/step - loss: 0.6073 - accuracy: 0.7250


[0.6072667837142944, 0.7250000238418579]